In [9]:
pip install pinecone pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 30.7 MB/s eta 0:00:00


In [7]:
from langchain_groq import ChatGroq

Model = ChatGroq(
    model="openai/gpt-oss-20b",
    api_key=GROQ_API_KEY
)

In [8]:
Model.invoke("who is vc of pakistan").content

'I’m not sure what you mean by “VC of Pakistan.”  \nCould you clarify? For example:\n\n- Are you asking about the Vice‑Chancellor of a specific university in Pakistan?  \n- Are you referring to a prominent venture capitalist (VC) based in Pakistan?  \n- Or something else entirely?\n\nLet me know so I can give you the most accurate answer.'

In [ ]:
!pip install pymupdf langchain_huggingface pinecone

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader,DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone

In [10]:
def load_pdfs_from_directory(dir_path):
    """
    Load all PDF files from a directory using PyMuPDFLoader.

    Args:
        directory_path (str): Path to the directory containing PDFs
        recursive (bool): Whether to load PDFs recursively

    Returns:
        List[Document]: List of LangChain Document objects (page-wise)
    """

    loader = DirectoryLoader(
        path=dir_path,
        glob="**/*.pdf",
        loader_cls=PyMuPDFLoader,
        show_progress=True
    )

    documents = loader.load()
    return documents


In [11]:
path = r"/content/Data"
docs = load_pdfs_from_directory(dir_path=path)

100%|██████████| 1/1 [00:08<00:00,  8.21s/it]


In [12]:
len(docs)   #mean there are 637 page in the book

637

In [13]:
docs[3]

Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': '', 'creationdate': '2004-12-18T17:00:02-05:00', 'source': '/content/Data/Medical_book.pdf', 'file_path': '/content/Data/Medical_book.pdf', 'total_pages': 637, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2004-12-18T16:15:31-06:00', 'trapped': '', 'modDate': "D:20041218161531-06'00'", 'creationDate': "D:20041218170002-05'00'", 'page': 3}, page_content='STAFF\nJacqueline L. Longe, Project Editor\nDeirdre S. Blanchfield, Associate Editor\nChristine B. Jeryan, Managing Editor\nDonna Olendorf, Senior Editor\nStacey Blachford, Associate Editor\nKate Kretschmann, Melissa C. McDade, Ryan\nThomason, Assistant Editors\nMark Springer, Technical Specialist\nAndrea Lopeman, Programmer/Analyst\nBarbara J. Yarrow, Manager, Imaging and Multimedia\nContent\nRobyn V. Young, Project Manager, Imaging and\nMultimedia Content\nDean Dauphinais, Senior Editor, Imaging and\nMultimedia Content\nK

In [15]:
from typing import List, Dict, Any
from langchain_core.documents import Document


def filtered_docs(docs:List[Document])->List[Dict[str,Any]]:
  filterd_docs = []
  for doc in docs:
    filterd_docs.append({
        "page_content":doc.page_content,
        "metadata":doc.metadata['source'],
        "page":doc.metadata['page']
    })
  return filterd_docs

In [16]:
docx = filtered_docs(docs)

In [17]:
type(docx)

list

In [18]:
def chunks_and_overlap(docs: list) -> list:
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100
    )
    return splitter.split_documents(docs)


In [19]:
chunks = chunks_and_overlap(docs)

In [20]:
len(chunks)

3128

In [21]:
def download_embeddings_models():
  embeddings = HuggingFaceEmbeddings(
      model="sentence-transformers/all-MiniLM-L6-v2",
  )
  return embeddings

In [22]:
embedding_model = download_embeddings_models()

In [23]:
emb = embedding_model.embed_query("Hey cheers! w3 got it")
emb

[-0.0923093855381012,
 0.0010174219496548176,
 -0.05877864360809326,
 0.025660475715994835,
 0.08164429664611816,
 -0.01278161071240902,
 0.10265175253152847,
 -0.05794999748468399,
 -0.039674289524555206,
 0.01850374974310398,
 0.046412527561187744,
 -0.04439261183142662,
 -0.03935921937227249,
 0.008393942378461361,
 -0.023489495739340782,
 0.03819684311747551,
 -0.05669350549578667,
 -0.05142394080758095,
 -0.06349252909421921,
 0.010298720560967922,
 0.018451858311891556,
 -0.05961323529481888,
 0.05787524953484535,
 -0.018127335235476494,
 0.009382067248225212,
 0.05564713478088379,
 -0.022222215309739113,
 0.07243376970291138,
 -0.018938671797513962,
 -0.018963035196065903,
 0.0132875582203269,
 -0.005670856684446335,
 0.028545206412672997,
 -0.02126770094037056,
 0.021311331540346146,
 -0.041626088321208954,
 -0.01484192069619894,
 0.026682276278734207,
 -0.030080480501055717,
 0.08758920431137085,
 -0.009658359922468662,
 0.029663601890206337,
 -0.02493264153599739,
 0.04922214

In [24]:
len(emb)

384

In [25]:
pc = Pinecone(api_key=PINECONE_API_KEY)

In [36]:
from pinecone import ServerlessSpec

index_name = "medical-chatbot"
indexes = [i["name"] for i in pc.list_indexes()]
if index_name not in indexes:
    pc.create_index(
        index_name,
        dimension = 384,
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region="us-east-1"
        )
    )

    print("Index has been created guys congrats.")
index = pc.Index(index_name)

In [27]:
!pip install langchain_pinecone

In [28]:
from langchain_pinecone import PineconeVectorStore

In [37]:
vectorstore = PineconeVectorStore(
    index=index,
    embedding=embedding_model
)

In [38]:
type(chunks)

list

In [39]:
vectorstore.add_documents(chunks)

['a1d3c7f5-74fc-4146-859a-0cd6138a353a',
 '41172e47-e00d-4817-9b48-797e6669f450',
 '9385b399-e8bd-4bda-9084-ffbab712638d',
 '09947ead-e490-400c-aed6-ca819963b24e',
 '321f449f-3e05-4df7-97f7-735e3e70aeaa',
 'c72d2759-d223-4a73-96ba-9806835d87f3',
 '4784d429-cc06-40f8-ae48-060c834ffd7d',
 '82c93ed1-33e8-4244-a748-16521610a953',
 'd75d6a22-992d-467a-9f14-b29ab738019b',
 '0252dd87-95b5-4fc0-af75-c16d93386a8d',
 '14841694-e7a0-425b-ae1e-c2c341ee5719',
 'b7805ec3-5157-4a3d-be35-c75f6d7af798',
 'e9103d86-4628-40ec-a196-00c105a63c86',
 '9723ee27-82a9-4f88-84fe-102c162f1701',
 '74322c4e-3882-417a-87ff-f424186cc687',
 '5ce8c850-c588-4eef-991f-9016091d90b1',
 '6d8c82d6-878e-47fa-9aa2-10e1a6a2f2a3',
 'a7f4ac23-bee7-479e-a4ce-35e02a0d0243',
 '8bf46f20-26a4-476c-a4eb-cfb4452b2d74',
 'f3b0c1b3-85c9-45f3-8fe2-0d40290ab406',
 'c5ddb166-b558-4a88-bb17-b57e43915e0d',
 '2a2f6842-0e7d-4f6b-b362-fe3641ba08e3',
 '1c37ca5a-7fb1-4449-9ae3-4d59c928f371',
 'd4cf8ae6-7bff-4eed-86ae-293f458bf0fa',
 '306e544a-1bb7-

In [44]:
new_data = Document(
    page_content="Hey my name is Waris. I am studying at NUML University Islamabad.",
    metadata={
        "source": "self",
        "page": 0
    }
)

In [45]:
vectorstore.add_documents([new_data])

['381a65fb-36ff-462b-9258-718df24e1351']

In [46]:
retriver = vectorstore.as_retriever(search_kwargs={"k": 3})

In [48]:
retriver.invoke("what is waris")[0].page_content

'Hey my name is Waris. I am studying at NUML University Islamabad.'

In [56]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

In [57]:
def get_sqlite_memory(session_id: str):
    return SQLChatMessageHistory(
        session_id=session_id,
        connection_string="sqlite:///chat_memory.db"
    )

In [70]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


prompt = ChatPromptTemplate.from_template("""
You are a helpful assistant.
Answer the question ONLY using the context below.
If the answer is not in the context, say "I don't know".

Context:
{context}

Question:
{question}
""")

chain = ({
    "context": retriver ,
    "question": RunnablePassthrough()
}
| prompt | Model | StrOutputParser())

In [71]:
chain = ({
    "context":retriver,
    "question":RunnablePassthrough(),
}) | prompt | Model | StrOutputParser()

In [74]:
chain.invoke("Noninvasive stimulators")

'Noninvasive stimulators use external electromagnetic coils placed on either side of the fracture. The coils are held in place by a strap or cuff, and their correct positioning is usually confirmed by X‑rays. They generate a pulsating electromagnetic field that stimulates bone growth, and the patient is responsible for maintaining the prescribed treatment schedule.'